# Packages & Connections

In [8]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install cassandra-driver

Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory

CASSANDRA_HOST= '185.185.126.143'
CASSANDRA_PORT= '9742'

auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster(contact_points=[CASSANDRA_HOST], port=CASSANDRA_PORT,
    auth_provider=auth_provider)

# Chart Data 

In [110]:
session = cluster.connect('yelp')
session.row_factory = dict_factory

sql_query = "SELECT * FROM yelp.user_review;"

df = pd.DataFrame()



for row in session.execute(sql_query):
    df = df.append(pd.DataFrame(row, index=[0]))

df = df.reset_index(drop=True).fillna(pd.np.nan)

print(df)

       date_only date_hour date_minute  stars               timestamp  \
0     2017-12-01         0           0    NaN 2021-07-30 11:51:00.004   
1     2017-12-01         0           1    NaN 2021-07-30 11:51:00.004   
2     2017-12-01         0          10    NaN 2021-07-30 11:52:00.003   
3     2017-12-01         0          11    NaN 2021-07-30 11:52:00.003   
4     2017-12-01         0          12    NaN 2021-07-30 11:52:00.003   
...          ...       ...         ...    ...                     ...   
1007  2017-12-01         9          44    NaN 2021-07-30 12:06:00.008   
1008  2017-12-01         9          45    NaN 2021-07-30 12:06:00.008   
1009  2017-12-01         9          47    NaN 2021-07-30 12:06:00.008   
1010  2017-12-01         9          52    NaN 2021-07-30 12:06:00.008   
1011  2017-12-01         9          59    NaN 2021-07-30 12:06:00.008   

      total_business  total_reviews  total_users  
0                  2              2            2  
1                  3 

/tmp/ipykernel_9176/1591178207.py:13: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



In [111]:
df["period"] = df['date_hour']+ ':' + df["date_minute"]
df['date_hour'] = pd.to_numeric(df['date_hour'])
df['date_minute'] = pd.to_numeric(df['date_minute'])
df["period"] = df["period"].apply(str)

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012 entries, 0 to 1011
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date_only       1012 non-null   object        
 1   date_hour       1012 non-null   int64         
 2   date_minute     1012 non-null   int64         
 3   stars           0 non-null      float64       
 4   timestamp       1012 non-null   datetime64[ns]
 5   total_business  1012 non-null   int64         
 6   total_reviews   1012 non-null   int64         
 7   total_users     1012 non-null   int64         
 8   period          1012 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(5), object(2)
memory usage: 71.3+ KB


In [113]:
df2=df.groupby('date_hour', as_index=False)['total_reviews'].sum()
df2=df2.sort_values('date_hour', ascending=True) 

In [114]:
import plotly.express as px

fig = px.line(df2, x=['date_hour'], y="total_reviews", title='title')
fig.show()

In [115]:
df2=df.groupby(['date_hour','date_minute'], as_index=False)['total_reviews'].sum()
df2=df2.sort_values(['date_hour','date_minute'], ascending=True) 
df2

,date_hour,date_minute,total_reviews
0,0,0,2
1,0,1,3
2,0,2,3
3,0,3,2
4,0,4,1
...,...,...,...
1007,23,55,2
1008,23,56,4
1009,23,57,2
1010,23,58,2


In [120]:
df2=df.groupby(['period'], as_index=False)['total_reviews'].sum()
df2=df2.sort_index(ascending=True) 
df2

,period,total_reviews
0,0:0,2
1,0:1,3
2,0:10,3
3,0:11,3
4,0:12,2
...,...,...
1007,9:44,1
1008,9:45,1
1009,9:47,2
1010,9:52,1


In [121]:
import plotly.express as px

fig = px.line(df2, x=['period'], y="total_reviews", title='title')
fig.show()